## InMemoryHistory

illufly 支持基于内存、文件的数据持久化。<br>
你可以继承 BaseHisotry 完成 redis、mongodb 等其他数据介质的持久化。

- 内存 InMemoryHistory
- 文件 LocalFileHistory

### 空的历史

In [12]:
from illufly.chat import ChatQwen

chat = ChatQwen(name="qwen")
chat.thread_ids

[]

### 写入历史对话

In [2]:
chat("给我写一首2句儿歌")
chat.memory

[USER] 给我写一首2句儿歌
小星星，亮晶晶，  
天上闪烁数不清。


[{'role': 'user', 'content': '给我写一首2句儿歌'},
 {'role': 'assistant', 'content': '小星星，亮晶晶，  \n天上闪烁数不清。'}]

In [3]:
chat.thread_ids

['992197-9335-0000']

In [4]:
chat("给我写一首2句儿歌", new_chat=True)
chat.memory

[USER] 给我写一首2句儿歌
小星星眨眼睛，月亮妈妈笑嘻嘻。


[{'role': 'user', 'content': '给我写一首2句儿歌'},
 {'role': 'assistant', 'content': '小星星眨眼睛，月亮妈妈笑嘻嘻。'}]

In [5]:
chat.thread_ids

['992197-9335-0000', '992200-5999-0001']

### 加载历史对话

In [6]:
chat.thread_id
chat.memory

[{'role': 'user', 'content': '给我写一首2句儿歌'},
 {'role': 'assistant', 'content': '小星星眨眼睛，月亮妈妈笑嘻嘻。'}]

In [7]:
chat.history.memory

{'992197-9335-0000': [{'role': 'user', 'content': '给我写一首2句儿歌'},
  {'role': 'assistant', 'content': '小星星，亮晶晶，  \n天上闪烁数不清。'}],
 '992200-5999-0001': [{'role': 'user', 'content': '给我写一首2句儿歌'},
  {'role': 'assistant', 'content': '小星星眨眼睛，月亮妈妈笑嘻嘻。'}]}

In [8]:
chat.load_memory(-2)
chat.memory

[{'role': 'user', 'content': '给我写一首2句儿歌'},
 {'role': 'assistant', 'content': '小星星，亮晶晶，  \n天上闪烁数不清。'}]

### 找回历史连续对话

In [10]:
chat("星星能改成月亮？")
chat.memory

[USER] 星星能改成月亮？
当然可以，改后的儿歌如下：

月亮姐姐眨眼睛，月亮妈妈笑嘻嘻。 

不过，这样修改后，第一句中的“眨眼睛”可能不太适合月亮，因为通常我们会用“眨眼睛”来形容星星。如果你不介意的话，我可以再调整一下，让儿歌更自然一些：

月亮姐姐挂空中，月亮妈妈笑嘻嘻。


[{'role': 'user', 'content': '给我写一首2句儿歌'},
 {'role': 'assistant', 'content': '小星星眨眼睛，月亮妈妈笑嘻嘻。'},
 {'role': 'user', 'content': '星星能改成月亮？'},
 {'role': 'assistant',
  'content': '当然可以，改后的儿歌如下：\n\n月亮姐姐眨眼睛，月亮妈妈笑嘻嘻。 \n\n不过，这样修改后，第一句中的“眨眼睛”可能不太适合月亮，因为通常我们会用“眨眼睛”来形容星星。如果你不介意的话，我可以再调整一下，让儿歌更自然一些：\n\n月亮姐姐挂空中，月亮妈妈笑嘻嘻。'}]

## LocalFileHistory

### 空的历史

In [1]:
from illufly.chat import ChatQwen
from illufly.io import LocalFileHistory

chat = ChatQwen(name="qwen", history=LocalFileHistory())
chat.thread_ids

['996713-7855-0000', '997004-4538-0001']

### 写入历史对话

In [2]:
chat("给我写一首2句儿歌")
chat.memory

[USER] 给我写一首2句儿歌
小星星，亮晶晶，  
天上闪烁数不清。


[{'role': 'user', 'content': '给我写一首2句儿歌'},
 {'role': 'assistant', 'content': '小星星，亮晶晶，  \n天上闪烁数不清。'}]

In [3]:
import tempfile
tempfile.gettempdir()

'/var/folders/f5/rlf27f4n6wzc_k4x7y4vzm5h0000gn/T'

In [4]:
chat.thread_ids

['996713-7855-0000', '997004-4538-0001']

In [4]:
chat("给我写一首2句儿歌", new_chat=True)
chat.memory

[USER] 给我写一首2句儿歌
小星星，亮晶晶，  
天上地上都是你。


[{'role': 'user', 'content': '给我写一首2句儿歌'},
 {'role': 'assistant', 'content': '小星星，亮晶晶，  \n天上地上都是你。'}]

In [5]:
chat.thread_ids

['995615-9748-0000', '995618-0659-0001']

### 加载历史对话

In [6]:
chat.thread_id
chat.memory

[{'role': 'user', 'content': '给我写一首2句儿歌'},
 {'role': 'assistant', 'content': '小星星，亮晶晶，  \n天上地上都是你。'}]

In [7]:
chat.load_memory(-2)
chat.memory

[{'role': 'user', 'content': '给我写一首2句儿歌'},
 {'role': 'assistant', 'content': '小星星，亮晶晶，  \n天上眨眼数不清。'}]

## 扩展持久化子类

参考内存持久化子类。

In [ ]:
import os
import json
import copy
from typing import Union, List
from illufly.types import BaseHistory

class InMemoryHistory(BaseHistory):
    """基于内存的记忆管理"""

    def __init__(self, memory: dict = {}, **kwargs):
        super().__init__(**kwargs)
        self.memory = memory or {}

    def last_thread_id_count(self):
        all_thread_ids = self.list_threads()
        if all_thread_ids:
            ids = all_thread_ids[-1].split("-")
            return int(ids[-1]) + 1
        else:
            return 0

    # 列举所有记忆线
    def list_threads(self):
        return sorted(self.memory.keys())

    def save_memory(self, thread_id: str, memory: List[dict]):
        self.memory[thread_id] = copy.deepcopy(memory)

    def load_memory(self, thread_id: Union[str, int] = None):
        """
        加载记忆。

        如果 thread_id 是字符串，则直接加载指定线程的记忆；
        如果 thread_id 是整数，则将其当作索引，例如 thread_id=-1 表示加载最近一轮对话的记忆。
        """
        _thread_id = thread_id
        if isinstance(thread_id, str):
            return _thread_id, self.memory.get(thread_id, [])
        elif isinstance(thread_id, int):
            all_threads = self.list_threads()
            if all_threads:
                _thread_id = all_threads[thread_id]
                return _thread_id, self.memory.get(_thread_id, [])

        return _thread_id, []